In [1]:
import os

import cv2
import pandas as pd
import numpy as np
import random

from skimage.feature import local_binary_pattern

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score

from sklearn.svm import SVC

import xgboost as xgb


<ipython-input-1-6fe4e1c89cb2>:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was too old on your system - pyarrow 10.0.1 is the current minimum supported version as of this release.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Paths are initialized

In [3]:
#wikiart (large dataset) will be added later

kaggle_train_path = "../data/train"
kaggle_test_path = "../data/test"

features_path = "../features"

kaggle_painter_names = os.listdir(kaggle_train_path)

In [4]:
painter_ids = {}
id = 0
for painter in kaggle_painter_names:
    painter_ids[painter] = id
    id += 1

Feature 1: Average of HOG

In [ ]:
#Computing HOG over an image
def compute_hog(image):
    hog_desc = cv2.HOGDescriptor()
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hog_features = []

    for channel in range(hsv_image.shape[2]): #computing for HSV channels
        hog_features_channel = hog_desc.compute(hsv_image[:, :, channel])
        hog_features.append(hog_features_channel)

    return np.concatenate(hog_features)

#Computing average of HOG for each painter's paintings distinctively
for painter in kaggle_painter_names:
    paintings_path = os.path.join(kaggle_train_path, painter)
    images_list = [image for image in os.listdir(paintings_path)]

    hog_features_list = []
    for image_name in images_list:
        image_path = os.path.join(paintings_path, image_name)
        image = cv2.imread(image_path)
        hog_features = compute_hog(image)
        hog_features_list.append(hog_features)

    hog_features_array = np.array(hog_features_list)
    avg_hog_features = np.mean(hog_features_array, axis=0)

Feature 2: Average of LBP

In [9]:
lbp_avg_path = "../features/lbp_avg.txt"

In [20]:
def compute_lbp(image):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    gray_image = hsv_image[:, :, 2] #to obtain texture information

    radius = 3
    n_points = 8 * radius
    features = local_binary_pattern(gray_image, n_points, radius, method="uniform")
    lbp_hist, _ = np.histogram(features.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
    lbp_hist = lbp_hist.astype(float)
    lbp_hist /= np.sum(lbp_hist)
    return lbp_hist

In [ ]:
average_lbp_features = {}

with open(lbp_avg_path, "w") as lbp_avg:
    #Computing average of LBP for each painter's paintings distinctively
    for painter in kaggle_painter_names:
        lbp_avg.write(painter)
        lbp_avg.write("\n")

        paintings_path = os.path.join(kaggle_train_path, painter)
        images_list = [image for image in os.listdir(paintings_path)]
        painter_lbp_features = []

        for image_name in images_list:
            image_path = os.path.join(paintings_path, image_name)
            image = cv2.imread(image_path)
            lbp_feature = compute_lbp(image)
            painter_lbp_features.append(lbp_feature)

        average_lbp_features[painter] = np.mean(painter_lbp_features, axis=0)
        np.savetxt(lbp_avg, [average_lbp_features[painter]], fmt="%.6f")

Feature 3: Standard Deviation of HOG

In [ ]:
#Computing standard deviation of HOG for each painter's paintings distinctively
for painter in kaggle_painter_names:
    paintings_path = os.path.join(kaggle_train_path, painter)
    images_list = [image for image in os.listdir(paintings_path)]

    hog_features_list = []
    for image_name in images_list:
        image_path = os.path.join(paintings_path, image_name)
        image = cv2.imread(image_path)
        hog_features = compute_hog(image)
        hog_features_list.append(hog_features)

    hog_features_array = np.array(hog_features_list)
    avg_hog_features = np.std(hog_features_array, axis=0)

Feature 4: Standard Deviation of LBP

In [10]:
lbp_std_path = "../features/lbp_std.txt"

In [ ]:
std_lbp_features = {}

with open(lbp_std_path, "w") as lbp_std:
    #Computing standard deviation of LBP for each painter's paintings distinctively
    for painter in kaggle_painter_names:
        lbp_std.write(painter)
        lbp_std.write("\n")

        paintings_path = os.path.join(kaggle_train_path, painter)
        images_list = [image for image in os.listdir(paintings_path)]
        painter_lbp_features = []

        for image_name in images_list:
            image_path = os.path.join(paintings_path, image_name)
            image = cv2.imread(image_path)
            lbp_feature = compute_lbp(image)
            painter_lbp_features.append(lbp_feature)

        std_lbp_features[painter] = np.std(painter_lbp_features, axis=0)
        np.savetxt(lbp_std, [std_lbp_features[painter]], fmt="%.6f")

Feature 5: Average of Gray Histogram

In [11]:
gray_hist_avg_path = "../features/gray_hist_avg.txt"

In [21]:
def compute_gray_histogram(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hist, _ = np.histogram(gray_image.ravel(), bins=256, range=[0,256])
    return hist

In [ ]:
avg_gray_hist = {}

with open(gray_hist_avg_path, "w") as gray_hist_avg:
    for painter in kaggle_painter_names:
        gray_hist_avg.write(painter)
        gray_hist_avg.write("\n")

        paintings_path = os.path.join(kaggle_train_path, painter)
        painter_gray_hist = []

        for image_name in os.listdir(paintings_path):
            image_path = os.path.join(paintings_path, image_name)
            image = cv2.imread(image_path)
            gray_hist = compute_gray_histogram(image)
            painter_gray_hist.append(gray_hist)

        avg_gray_hist[painter] = np.mean(painter_gray_hist, axis=0)
        np.savetxt(gray_hist_avg, [avg_gray_hist[painter]])

Feature 6: Average of Image Edges Histogram

In [12]:
edges_hist_avg_path = "../features/edges_hist_avg.txt"

In [22]:
def compute_edge_histogram(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray_image, 50, 150)
    hist, _ = np.histogram(edges.ravel(), bins=256, range=[0,256])
    return hist

In [ ]:
avg_edge_hist = {}

with open(edges_hist_avg_path, "w") as edges_hist_avg:
    for painter in kaggle_painter_names:
        edges_hist_avg.write(painter)
        edges_hist_avg.write("\n")

        paintings_path = os.path.join(kaggle_train_path, painter)
        painter_edge_hist = []

        for image_name in os.listdir(paintings_path):
            image_path = os.path.join(paintings_path, image_name)
            image = cv2.imread(image_path)
            edge_hist = compute_edge_histogram(image)
            painter_edge_hist.append(edge_hist)

        avg_edge_hist[painter] = np.mean(painter_edge_hist, axis=0)
        np.savetxt(edges_hist_avg, [avg_edge_hist[painter]], fmt="%.6f")

Feature 7: Means of Image Edges Histogram

In [13]:
edges_feat_mean_path = "../features/edges_feat_mean.txt"

In [23]:
def compute_edge_mean(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray_image, 50, 150)
    edge_mean = np.mean(edges)
    return edge_mean

In [ ]:
edge_features_means = {}

with open(edges_feat_mean_path, "w") as edges_feat_mean:
    for painter in kaggle_painter_names:
        edges_feat_mean.write(painter)
        edges_feat_mean.write("\n")

        paintings_path = os.path.join(kaggle_train_path, painter)
        painter_edge_means = []

        for image_name in os.listdir(paintings_path):
            image_path = os.path.join(paintings_path, image_name)
            image = cv2.imread(image_path)
            edge_mean = compute_edge_mean(image)
            painter_edge_means.append(edge_mean)

        edge_features_means[painter] = np.mean(painter_edge_means)
        np.savetxt(edges_feat_mean, [edge_features_means[painter]], fmt="%.6f")

Feature 8: Average of Histogram for S Component From HSV Image

In [14]:
s_hsv_avg_path = "../features/s_hsv_avg.txt"

In [24]:
def compute_s_histogram(image):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    s_component = hsv_image[:,:,1]
    hist, _ = np.histogram(s_component.ravel(), bins=256, range=[0,256])
    return hist

In [ ]:
avg_s_hist = {}

with open(s_hsv_avg_path, "w") as s_hsv_avg:
    for painter in kaggle_painter_names:
        s_hsv_avg.write(painter)
        s_hsv_avg.write("\n")

        paintings_path = os.path.join(kaggle_train_path, painter)
        painter_s_hist = []
        
        for image_name in os.listdir(paintings_path):
            image_path = os.path.join(paintings_path, image_name)
            image = cv2.imread(image_path)
            s_hist = compute_s_histogram(image)
            painter_s_hist.append(s_hist)
            
        avg_s_hist[painter] = np.mean(painter_s_hist, axis=0)
        np.savetxt(s_hsv_avg, [avg_s_hist[painter]], fmt="%.6f")

Feature 9: Standard Deviation of Histogram for S Component From HSV Image

In [15]:
s_hsv_std_path = "../features/s_hsv_std.txt"

In [ ]:
std_s_hist = {}

with open(s_hsv_std_path, "w") as s_hsv_std:
    for painter in kaggle_painter_names:
        s_hsv_std.write(painter)
        s_hsv_std.write("\n")

        paintings_path = os.path.join(kaggle_train_path, painter)
        painter_s_hist = []
        
        for image_name in os.listdir(paintings_path):
            image_path = os.path.join(paintings_path, image_name)
            image = cv2.imread(image_path)
            s_hist = compute_s_histogram(image)
            painter_s_hist.append(s_hist)
            
        std_s_hist[painter] = np.std(painter_s_hist, axis=0)
        np.savetxt(s_hsv_std, [std_s_hist[painter]], fmt="%.6f")

Feature 10: Average of Histogram for B Component From RGB Image

In [16]:
b_rgb_avg_path = "../features/b_rgb_avg.txt"

In [25]:
def compute_b_histogram(image):
    b, g, r = cv2.split(image)
    hist, _ = np.histogram(b.ravel(), bins=256, range=[0,256])
    return hist

In [ ]:
avg_b_hist = {}

with open(b_rgb_avg_path, "w") as b_rgb_avg:
    for painter in kaggle_painter_names:
        b_rgb_avg.write(painter)
        b_rgb_avg.write("\n")

        paintings_path = os.path.join(kaggle_train_path, painter)
        painter_b_hist = []
        
        for image_name in os.listdir(paintings_path):
            image_path = os.path.join(paintings_path, image_name)
            image = cv2.imread(image_path)
            b_hist = compute_b_histogram(image)
            painter_b_hist.append(s_hist)
            
        avg_b_hist[painter] = np.mean(painter_b_hist, axis=0)
        np.savetxt(b_rgb_avg, [avg_b_hist[painter]], fmt="%.6f")

Feature 11: Standard Deviation of Histogram for B Component From RGB Image

In [17]:
b_rgb_std_path = "../features/b_rgb_std.txt"

In [ ]:
std_b_hist = {}

with open(b_rgb_std_path, "w") as b_std_avg:
    for painter in kaggle_painter_names:
        b_std_avg.write(painter)
        b_std_avg.write("\n")

        paintings_path = os.path.join(kaggle_train_path, painter)
        painter_b_hist = []
        
        for image_name in os.listdir(paintings_path):
            image_path = os.path.join(paintings_path, image_name)
            image = cv2.imread(image_path)
            b_hist = compute_b_histogram(image)
            painter_b_hist.append(s_hist)
            
        std_b_hist[painter] = np.std(painter_b_hist, axis=0)
        np.savetxt(b_std_avg, [std_b_hist[painter]], fmt="%.6f")

Preparation of Test Data

In [7]:
#For the first way
def get_test_by_features():
    #epoch_num = 50
    test_file_paths = []

    """
    for i in range(epoch_num):
        random_painter = random.sample(kaggle_painter_names, 1)[0]
        painter_path = f"../data/test/{random_painter}/"
        painting_names = os.listdir(painter_path)
        image1_name, image2_name = random.sample(painting_names, 2)
        path1 = os.path.join(painter_path, image1_name)
        path2 = os.path.join(painter_path, image2_name)
        test_file_paths.append([path1, path2])
    """

    for painter in kaggle_painter_names:
        painter_path = f"../data/test/{painter}/"
        painting_names = os.listdir(painter_path)

        for i in range(len(painting_names)-1): #compute for each pair of each painter
            image1_name, image2_name = painting_names[i], painting_names[i+1]
            path1 = os.path.join(painter_path, image1_name)
            path2 = os.path.join(painter_path, image2_name)
            test_file_paths.append([path1, path2])

    X_test = []
    y_test = []

    for file_pair in test_file_paths:
        path1 = file_pair[0]
        path2 = file_pair[1]

        image1 = cv2.imread(path1)
        image2 = cv2.imread(path2)

        lbp_feature1 = compute_lbp(image1)
        lbp_feature2 = compute_lbp(image2)
        pair_lbp_features = []
        pair_lbp_features.append(lbp_feature1)
        pair_lbp_features.append(lbp_feature2)
        avg_lbp_features = np.mean(pair_lbp_features, axis=0)

        std_lbp_features = np.std(pair_lbp_features, axis=0)

        pair_gray_hist_features = []
        gray_hist1 = compute_gray_histogram(image1)
        gray_hist2 = compute_gray_histogram(image2)
        pair_gray_hist_features.append(gray_hist1)
        pair_gray_hist_features.append(gray_hist2)
        gray_hist_features = np.mean(pair_gray_hist_features, axis=0)

        pair_edge_features = []
        edge_hist1 = compute_edge_histogram(image1)
        edge_hist2 = compute_edge_histogram(image2)
        pair_edge_features.append(edge_hist1)
        pair_edge_features.append(edge_hist2)
        edge_hist_features = np.mean(pair_edge_features, axis=0)
        
        pair_edge_means = []
        edge_mean1 = compute_edge_mean(image1)
        edge_mean2 = compute_edge_mean(image2)
        pair_edge_means.append(edge_mean1)
        pair_edge_means.append(edge_mean2)
        edge_mean_features = np.mean(pair_edge_means, axis=0)
        
        pair_s_features = []
        s_hist1 = compute_s_histogram(image1)
        s_hist2 = compute_s_histogram(image2)
        pair_s_features.append(s_hist1)
        pair_s_features.append(s_hist2)
        avg_s_hist = np.mean(pair_s_features, axis=0)
        
        std_s_hist = np.std(pair_s_features, axis=0)

        pair_b_features = []
        b_hist1 = compute_b_histogram(image1)
        b_hist2 = compute_b_histogram(image2)
        pair_b_features.append(b_hist1)
        pair_b_features.append(b_hist2)
        avg_b_hist = np.mean(pair_b_features, axis=0)
        
        std_b_hist = np.std(pair_b_features, axis=0)

        pair_concatenated_features = []
        pair_concatenated_features.extend(avg_lbp_features)
        pair_concatenated_features.extend(std_lbp_features)
        pair_concatenated_features.extend(gray_hist_features)
        pair_concatenated_features.extend(edge_hist_features)
        pair_concatenated_features.append(edge_mean_features)
        pair_concatenated_features.extend(avg_s_hist)
        pair_concatenated_features.extend(std_s_hist)
        pair_concatenated_features.extend(avg_b_hist)
        pair_concatenated_features.extend(std_b_hist)
        pair_concatenated_features = np.array(pair_concatenated_features)

        X_test.append(pair_concatenated_features)
        #print("len:", len(pair_concatenated_features))
        painter = os.path.basename(os.path.dirname(path1))
        y_test.append(painter_ids[painter])

    return X_test, y_test

In [ ]:
#For the second way
def get_basic_features(test_image_path):
    test_image = cv2.imread(test_image_path)
    test_image = cv2.resize(test_image, (500, 500))

    test_lbp = compute_lbp(test_image)
    test_gray_hist = compute_gray_histogram(test_image)
    test_edge_hist = compute_edge_histogram(test_image)
    test_edge_mean = compute_edge_mean(test_image)
    test_s_hist = compute_s_histogram(test_image)
    test_b_hist = compute_b_histogram(test_image)

    features_for_test = []
    features_for_test.extend(test_lbp) 
    features_for_test.extend(test_gray_hist) 
    features_for_test.extend(test_edge_hist) 
    features_for_test.append(test_edge_mean)
    features_for_test.extend(test_s_hist)
    features_for_test.extend(test_s_hist)
    features_for_test.extend(test_b_hist)
    features_for_test.extend(test_b_hist)

    return features_for_test

# First Way

Train Data 1: Using the computed features (including averages, standard deviations etc.)

In [18]:
concatenated_features_path = "../features/concatenated_features.txt"

concatenated_features = {}

#Concatenate features for each painter
for painter in kaggle_painter_names:
    painter_features = []

    with open(lbp_avg_path, "r") as lbp_avg_file:
        for line in lbp_avg_file:
            if line.strip() == painter:
                next_line = lbp_avg_file.readline().strip()
                average_lbp_features = np.fromstring(next_line, sep=" ")
                painter_features.extend(average_lbp_features)
                break

    with open(lbp_std_path, "r") as lbp_std_file:
        for line in lbp_std_file:
            if line.strip() == painter:
                next_line = lbp_std_file.readline().strip()
                std_lbp_features = np.fromstring(next_line, sep=" ")
                painter_features.extend(std_lbp_features)
                break

    with open(gray_hist_avg_path, "r") as gray_hist_file:
        for line in gray_hist_file:
            if line.strip() == painter:
                next_line = gray_hist_file.readline().strip()
                gray_hist_avg = np.fromstring(next_line, sep=" ")
                painter_features.extend(gray_hist_avg)
                break

    with open(edges_hist_avg_path, "r") as edges_hist_file:
        for line in edges_hist_file:
            if line.strip() == painter:
                next_line = edges_hist_file.readline().strip()
                edges_hist_avg = np.fromstring(next_line, sep=" ")
                painter_features.extend(edges_hist_avg)
                break

    with open(edges_feat_mean_path, "r") as edges_feat_mean_file:
        for line in edges_feat_mean_file:
            if line.strip() == painter:
                next_line = edges_feat_mean_file.readline().strip()
                edge_feat_mean = np.fromstring(next_line, sep=" ")
                painter_features.extend(edge_feat_mean)
                break

    with open(s_hsv_avg_path, "r") as s_hsv_avg_file:
        for line in s_hsv_avg_file:
            if line.strip() == painter:
                next_line = s_hsv_avg_file.readline().strip()
                s_hsv_avg = np.fromstring(next_line, sep=" ")
                painter_features.extend(s_hsv_avg)
                break

    with open(s_hsv_std_path, "r") as s_hsv_std_file:
        for line in s_hsv_std_file:
            if line.strip() == painter:
                next_line = s_hsv_std_file.readline().strip()
                s_hsv_std = np.fromstring(next_line, sep=" ")
                painter_features.extend(s_hsv_std)
                break

    with open(b_rgb_avg_path, "r") as b_rgb_avg_file:
        for line in b_rgb_avg_file:
            if line.strip() == painter:
                next_line = b_rgb_avg_file.readline().strip()
                b_rgb_avg = np.fromstring(next_line, sep=" ")
                painter_features.extend(b_rgb_avg)
                break

    with open(b_rgb_std_path, "r") as b_rgb_std_file:
        for line in b_rgb_std_file:
            if line.strip() == painter:
                next_line = b_rgb_std_file.readline().strip()
                b_rgb_std = np.fromstring(next_line, sep=" ")
                painter_features.extend(b_rgb_std)
                break

    #Add the corresponding features for that painter
    concatenated_features[painter] = np.array(painter_features)

#Save concatenated features to the file
with open(concatenated_features_path, "w") as concatenated_file:
    for painter, features in concatenated_features.items():
        concatenated_file.write(painter + "\n")
        np.savetxt(concatenated_file, [features], fmt="%.6f")

Obtaining the test data's features

In [26]:
X_test_1, y_test_1 = get_test_by_features()

Train Data 1: Obtain the Data

In [27]:
X_train_1 = []
y_train_1 = []

for painter in kaggle_painter_names:
    y_train_1.append([painter_ids[painter]])

with open(concatenated_features_path, "r") as concatenated_file:
    lines = concatenated_file.readlines()
    for i in range(0, len(lines), 2):  #Every other line is a painter name
        painter_name = lines[i].strip()
        if painter_name in kaggle_painter_names:
            features = np.fromstring(lines[i + 1].strip(), sep=" ")
            X_train_1.append(features)

X_train_1 = np.array(X_train_1)
y_train_1 = np.array(y_train_1)

ML Model 1: MLP

In [28]:
mlp_model = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=500, random_state=42)

mlp_model.fit(X_train_1, y_train_1)
y_pred_mlp_1 = mlp_model.predict(X_test_1)

results_mlp_path_1 = "../results/results_mlp_1.txt"

accuracy_mlp_1 = accuracy_score(y_test_1, y_pred_mlp_1)
report_mlp_1 = classification_report(y_test_1, y_pred_mlp_1)

with open(results_mlp_path_1, "w") as results_mlp:
    results_mlp.write(report_mlp_1)
    results_mlp.write("\n")
    results_mlp.write(f"Accuracy: {accuracy_mlp_1}")

c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1101: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", le

ML Model 2: SVM

In [29]:
svm_model = SVC(kernel='linear', random_state=42)

svm_model.fit(X_train_1, y_train_1)
y_pred_svm_1 = svm_model.predict(X_test_1)

accuracy_svm_1 = accuracy_score(y_test_1, y_pred_svm_1)
report_svm_1 = classification_report(y_test_1, y_pred_svm_1)

results_svm_path_1 = "../results/results_svm_1.txt"

with open(results_svm_path_1, "w") as results_svm:
    results_svm.write(report_svm_1)
    results_svm.write("\n")
    results_svm.write(f"Accuracy: {accuracy_svm_1}")

c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1229: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\u

ML Model 3: XGBoost

In [30]:
xgb_model = xgb.XGBClassifier(random_state=42)

xgb_model.fit(X_train_1, y_train_1)
y_pred_xgb_1 = xgb_model.predict(X_test_1)

accuracy_xgb_1 = accuracy_score(y_test_1, y_pred_xgb_1)
report_xgb_1 = classification_report(y_test_1, y_pred_xgb_1)

results_xgb_path_1 = "../results/results_xgb_1.txt"

with open(results_xgb_path_1, "w") as results_xgb:
    results_xgb.write(report_xgb_1)
    results_xgb.write("\n")
    results_xgb.write(f"Accuracy: {accuracy_xgb_1}")

c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

# Second Way

Test Data's Features Obtaining for 2nd Way

In [ ]:
X_test_2 = []
y_test_2 = []

for painter in kaggle_painter_names:
    test_paintings_path = os.path.join(kaggle_test_path, painter)
    
    for test_image_name in os.listdir(test_paintings_path):
        test_image_path = os.path.join(test_paintings_path, test_image_name)
        test_features = get_basic_features(test_image_path)
        X_test_2.append(test_features)
        y_test_2.append(painter_ids[painter])

X_test_2 = np.array(X_test_2)
y_test_2 = np.array(y_test_2)

Train Data 2: Computing features excluding averages, standard deviations etc.

In [ ]:
X_train_2 = []
y_train_2 = []

for painter in kaggle_painter_names:
    train_paintings_path = os.path.join(kaggle_train_path, painter)
    
    for train_image_name in os.listdir(train_paintings_path):
        train_image_path = os.path.join(train_paintings_path, train_image_name)
        train_features = get_basic_features(train_image_path)
        X_train_2.append(train_features)
        y_train_2.append(painter_ids[painter])

X_train_2 = np.array(X_train_2)
y_train_2 = np.array(y_train_2)

ML Model 1: MLP

In [ ]:
mlp_model = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=500, random_state=42)

mlp_model.fit(X_train_2, y_train_2)
y_pred_mlp_2 = mlp_model.predict(X_test_2)

results_mlp_path_2 = "../results/results_mlp_2.txt"

accuracy_mlp_2 = accuracy_score(y_test_2, y_pred_mlp_2)
report_mlp_2 = classification_report(y_test_2, y_pred_mlp_2)

with open(results_mlp_path_2, "w") as results_mlp:
    results_mlp.write(report_mlp_1)
    results_mlp.write("\n")
    results_mlp.write(f"Accuracy: {accuracy_mlp_2}")

ML Model 2: SVM

In [ ]:
svm_model = SVC(kernel='linear', random_state=42)

svm_model.fit(X_train_2, y_train_2)
y_pred_svm_2 = svm_model.predict(X_test_2)

accuracy_svm_2 = accuracy_score(y_test_2, y_pred_svm_2)
report_svm_2 = classification_report(y_test_2, y_pred_svm_2)

results_svm_path_2 = "../results/results_svm_2.txt"

with open(results_svm_path_2, "w") as results_svm:
    results_svm.write(report_svm_2)
    results_svm.write("\n")
    results_svm.write(f"Accuracy: {accuracy_svm_2}")

ML Model 3: XGBoost

In [ ]:
xgb_model = xgb.XGBClassifier(random_state=42)

xgb_model.fit(X_train_2, y_train_2)
y_pred_xgb_2 = xgb_model.predict(X_test_2)

accuracy_xgb_2 = accuracy_score(y_test_2, y_pred_xgb_2)
report_xgb_2 = classification_report(y_test_2, y_pred_xgb_2)

results_xgb_path_2 = "../results/results_xgb_2.txt"

with open(results_xgb_path_2, "w") as results_xgb:
    results_xgb.write(report_xgb_2)
    results_xgb.write("\n")
    results_xgb.write(f"Accuracy: {accuracy_xgb_2}")